In [27]:
import numpy as np
import pandas as pd # Для работы с данными
import scipy.stats # При работе со статистикой
import matplotlib.pyplot as plt  # Библиотека для визуализации результатов 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [20]:
data = pd.read_csv('adult.csv') # датасет - https://www.kaggle.com/rio2016/olympic-games
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [17]:
data.info()
# кол-во пропущенных строк в сттолбцах "workclass" и "occupation" 2799. Исключаем 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [19]:
data_new = data[(data.workclass!='?') & (data.occupation!='?') & (data['native-country']!='?')]
data_new.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [18]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data_new[ [ 'age', 'occupation', 'income'] ]
# selectedColumns
# столбец sport является категориальной переменной
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями. one hot incoding
X = pd.get_dummies( selectedColumns, columns = [ 'occupation' ] ) # функция pandas one hot incoding преобразования 
# X
# столбец sex является целевой переменной, удаляем его из X
del X['income']
X.head()

,age,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,25,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,38,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,28,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,44,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,34,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [21]:
# целевая переменная (столбец income) снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [22]:
le.fit( data_new['income'] )

LabelEncoder()

In [23]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [24]:
le.transform( [ '<=50K', '>50K', '<=50K' ] )

array([0, 1, 0])

In [25]:
y = pd.Series( data = le.transform( data_new['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [28]:
model = LogisticRegression()

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) 

In [33]:
model.fit(X_train, y_train)
predictions = model.predict_proba( X_test )
predictions[:5]

C:\Users\snikulin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[0.93630307, 0.06369693],
       [0.88528841, 0.11471159],
       [0.72247551, 0.27752449],
       [0.8742056 , 0.1257944 ],
       [0.67068863, 0.32931137]])

In [34]:
model.predict(X_test)

array([0, 0, 0, ..., 0, 1, 0])

In [35]:
model.score(X_train, y_train)

0.7597921331232551

In [36]:
model.score(X_test, y_test)# метод возвращает значение коэффициента детерминации (оценка качества модели: сравниваем со средней моделью)

0.7634051962410171

In [37]:
from sklearn.pipeline import make_pipeline  # поток последовательных действий
from sklearn.preprocessing import StandardScaler

In [38]:
pipe_model = make_pipeline(StandardScaler(),LogisticRegression(max_iter=1000)) 

In [39]:
pipe_model.fit(X_train, y_train)
predictions = pipe_model.predict_proba( X_test )
predictions[:5]

array([[0.93857123, 0.06142877],
       [0.87955265, 0.12044735],
       [0.72267533, 0.27732467],
       [0.87457315, 0.12542685],
       [0.67077904, 0.32922096]])

In [40]:
pipe_model.predict(X_test)

array([0, 0, 0, ..., 0, 1, 0])

In [41]:
pipe_model.score(X_train, y_train)

0.7605937474085745

In [42]:
pipe_model.score(X_test, y_test) # результат аналогичный, что и в методе выше. Разницы нет.

0.7634051962410171

In [43]:
from sklearn.svm import SVC

In [44]:
# метод SVM с маштабированием.
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [45]:
clf.score(X_train, y_train)

0.7737789203084833

In [46]:
clf.score(X_test, y_test) # метод SVM с маштабирования дает нам рез-т выше чем pipeline.

0.7763405196241017

In [47]:
svc = SVC(gamma='auto') 

In [48]:
svc.fit(X_train, y_train)

SVC(gamma='auto')

In [50]:
svc.score(X_train, y_train)

0.7756032838543826

In [51]:
svc.score(X_test, y_test) # метод SVM без маштабирования дает нам самый высокий рез-т.

0.7765616362631288